Step 1: Establish Imports

In [1]:
import pandas as pd
import numpy as np

Teting Reading in Energystar Template

In [2]:
energystar_excel_file = "Empty_AAPS_Template.xlsx"
constellation_data_file = "AAPS_CONST.xlsx"

In [3]:
# this funtion performs data cleaning and manipulating tasks for the energystar file
# the goal of this is to have a copy of the energystar data as a dataframe and (more importantly) a list of all of the unique meter names
# from the energystar excel sheet. We will use this list of unique names later on
def read_energystar_data(filename):
    read_file = pd.read_excel(filename, sheet_name=None)
    # display(read_file.keys())
    # print("LENGTH: ", len(read_file.items()))
    es_data = read_file['Add Bills-Non Electric']
    es_data.columns = es_data.columns.str.replace('\n', ' ')

    # display(es_data.columns)

    # this gets the first part of the meter name before any commas (sometimes there would be a comma and unnessesary info after this)
    es_data["Meter Name (Pre-filled)"] = es_data["Meter Name (Pre-filled)"].str.split(",").str[0]
    # then we filter this so the name matches our naming convention
    es_data["Meter Name (Pre-filled)"] = es_data["Meter Name (Pre-filled)"][es_data["Meter Name (Pre-filled)"].str.contains("Constellation__RG-\d{8}__\d{10}") == True]

    # regex check on the colestllation ()

    # then we consolidate to get a unique list of all the meter names
    es_data_meter_names = es_data["Meter Name (Pre-filled)"].unique()
    return es_data, es_data_meter_names

In [4]:
energystar_data, energystar_data_meter_names = read_energystar_data(energystar_excel_file)
# display(energystar_data)
display(energystar_data_meter_names)

C:\Users\briel\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


array([nan, 'Constellation__RG-43035769__2420793405',
       'Constellation__RG-43035764__4020680140',
       'Constellation__RG-43035792__2859310035',
       'Constellation__RG-43035790__2429100312',
       'Constellation__RG-43035763__2480591620',
       'Constellation__RG-43035796__2859710142',
       'Constellation__RG-43035780__2477225928',
       'Constellation__RG-43035777__2146998387',
       'Constellation__RG-43035793__2849410219',
       'Constellation__RG-43035775__2156500195',
       'Constellation__RG-43035802__2859902146',
       'Constellation__RG-43035765__2480692729',
       'Constellation__RG-43035785__2488706136',
       'Constellation__RG-43035807__2161936262',
       'Constellation__RG-43035808__2411935688',
       'Constellation__RG-43035782__2428262178',
       'Constellation__RG-43035787__4248850022',
       'Constellation__RG-43035779__2427107785',
       'Constellation__RG-43035766__2420692744',
       'Constellation__RG-43035801__2829810387',
       'Constel

In [5]:
# this funtion performs data cleaning and manipulating tasks for the constellation file
def constellation_file_cleanup(file_name):
    const_excel = pd.read_excel(file_name)
    # use where function to say if reading is actual of estimated
    const_excel['Actual Or Estimated'] = np.where(const_excel['EndReadType'] == "Actual", 'No', 'Yes')

    # this function converts the constellation MeterNumber column into the Constellation__[CustomerId]__[MeterNumber] format
    def name_helper(row):
        row_id = str(row['CustomerId'])
        row_number = str(row['MeterNumber'])
        final_string = f"Constellation__{row_id}__{row_number}"
        return final_string
    const_excel['Const_Meter_ID'] = const_excel.apply(name_helper, axis=1)

    # this is a list of dfs, each df holds constellation data for a unique meter in constellation
    unique_meter_const = []
    for unique_meter_name in const_excel['MeterNumber'].unique():
        # display(unique_meter_name)
        temp_df = const_excel[const_excel['MeterNumber'] == unique_meter_name]
        # display(temp_df)  
        unique_meter_const.append(temp_df)

    return const_excel, unique_meter_const

In [6]:
constellation_excel, unique_meter_data_from_const = constellation_file_cleanup(constellation_data_file)
# display(constellation_excel)
# display(constellation_excel.columns)
# display(unique_meter_data_from_const[0])

This opens the excel file that we can edit directly:

In [7]:
# from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook

# this function creates an instance of an excel workbook where we can use openpyxl to edit the excel sheet directly
def create_es_excel_wb(file_name):
    es_workbook = load_workbook(filename=file_name)
    display(es_workbook.sheetnames)
    # sheet = es_workbook.active
    sheet = es_workbook["Add Bills-Non Electric"]
    return es_workbook, sheet

output_workbook, es_sheet = create_es_excel_wb(energystar_excel_file)

C:\Users\briel\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


['Instructions',
 'Add Bills-Electricity',
 'Add Bills-Non Electric',
 'Add Bills-Water',
 'Add Bills-IT Energy',
 'Add Bills-Flow',
 'Add Bills-On-Site',
 'Add Bills-Delivery',
 'Lists',
 'Validation',
 'Regular Waste Amounts',
 'Intermittent Waste',
 'Regular Waste-Container']

In [8]:
display(es_sheet)
display(es_sheet.title)

<Worksheet "Add Bills-Non Electric">

'Add Bills-Non Electric'

In [9]:
# for row in es_sheet.iter_rows( max_row=3):
#     display([cell.value for cell in row])
# display(constellation_excel.columns)

In [10]:
# unique_meter_data_from_const.reverse()
# display(unique_meter_data_from_const[0])
# display(energystar_data_meter_names)

In [11]:
# this funtion populates the energystar excel spreadsheet with the data from constellation
def populate_spreadsheet(const_dfs, es_meter_names, energystar_pop_sheet):
    meters_set = set()
    # this set will have all the unique meter names in the spreadsheet
    for df in const_dfs:
        # const_dfs is a list of dataframes. Each dataframe shows all of the constellation data from a unique meter
        # so looping through this list of dataframes lets us examine each unique meter from constellation at a time
        # and thus we will be adding this data to the energystar excel sheet with 1 meter from constellation's data at a time
        for row in df.iterrows():
            # row[0] is other information about the row, row[1] is the data itself
            row_data = row[1]

            if row_data['Const_Meter_ID'] not in es_meter_names:
                # if th constellation meter id is not in the list of energystar meter names, then we skip to the next dataframe
                # this means that there is a meter constellation took data on that is not in the energystar database
                continue
            # else:
            #     display(row_data)
            # display(row_data['ESPM_Meter_ID'])

            # this copies the data for the row from constellation into variables like meter_number
            meter_number = row_data['MeterNumber']
            customer_id = row_data['CustomerId']
            meter_string = f"Constellation__{customer_id}__{meter_number}"

            start_date = row_data['CycleStartDate']
            end_date = row_data['CycleEndDate']
            usage = row_data['EndReading'] - row_data['BeginningReading'] #feevolume or diff?
            meter_multiplier = row_data['MeterMultiplier']
            fee_volume = row_data['FeeVolume']
            total_charges = row_data['TotalCharges']
            actual_estimated = row_data['Actual Or Estimated']

            # start with index 1 because excel isn't 0-based
            insert_index = 1
            for row in energystar_pop_sheet.iter_rows():
                # display(row[4].value)
                
                # meter_name = str(row[4].value)
                meter_name = row[4].value
                # for each row in the energystar excel sheet, we first check to see if the energystar "Meter Name (Pre-filled)" column matches the constellation meter id
                if meter_name == row_data['Const_Meter_ID']:
                    # if we the energystar meter name matches the constllation meter name, then we found the right now in the energystar excel sheet and can now inserting the constellation data into this row
                    # display([cell.value for cell in row])
                    break
                else:
                    # if the name doesn't match, then we increase the insert index number because it means we need to look at the next row in the energystar table (the meter names didn't match, so we look at the next row in constellation)
                    insert_index += 1
            
            insert_index += 1
            # incrememnt the insert index by 1 to put our new row below the row with matching meter names
            energystar_pop_sheet.insert_rows(insert_index, amount=1)
 
            # now we enter data into this new row we just created
            # there are some rows that are energystar specific (not included in the constellation reports). These are columns A, C, D, F, and J in the excel sheet
            # for these we will simply copy the information from the row above, because we know the row above is a valid row that will be accepted into energystar
            # The data from constellation that we want to transfer (like start_date, end_date, free_volumne, etc.) we enter into the matching columns in energystar 
            # ("Start Date (Required)", "End Date (Required)", "Quantity (Required)", etc.)
            energystar_pop_sheet.cell(row=insert_index, column=1).value = energystar_pop_sheet[f"A{insert_index-1}"].value
            # energystar_pop_sheet.cell(row=insert_index, column=2).value = energystar_pop_sheet[f"B{insert_index-1}"].value
            energystar_pop_sheet.cell(row=insert_index, column=3).value = energystar_pop_sheet[f"C{insert_index-1}"].value
            energystar_pop_sheet.cell(row=insert_index, column=4).value = energystar_pop_sheet[f"D{insert_index-1}"].value

            # sheet.cell(row=insert_index, column=5).value = sheet[f"E{insert_index-1}"].value
            energystar_pop_sheet.cell(row=insert_index, column=5).value = meter_string
            energystar_pop_sheet.cell(row=insert_index, column=6).value = energystar_pop_sheet[f"F{insert_index-1}"].value
            energystar_pop_sheet.cell(row=insert_index, column=7).value = start_date
            energystar_pop_sheet.cell(row=insert_index, column=8).value = end_date
            energystar_pop_sheet.cell(row=insert_index, column=9).value = fee_volume
            energystar_pop_sheet.cell(row=insert_index, column=10).value = energystar_pop_sheet[f"J{insert_index-1}"].value
            energystar_pop_sheet.cell(row=insert_index, column=11).value = total_charges
            energystar_pop_sheet.cell(row=insert_index, column=12).value = actual_estimated

            meters_set.add(meter_string)

    return (meters_set, energystar_pop_sheet)

In [12]:
meters_set, es_sheet = populate_spreadsheet(unique_meter_data_from_const, energystar_data_meter_names, es_sheet)

In [13]:
# uncomment below if you want to write all of the meter names as a set and export the file:

# import json
# with open('meter_set.txt', 'w') as f:
#     f.write(str(meters_set))

In [14]:
# output_workbook.save(filename="old_meters_output_file.xlsx")

In [15]:
for row in es_sheet.iter_rows(min_row=1, max_row=3):
    display([cell.value for cell in row])
    # display(row[4].value)

['Meter ID\n(Pre-filled)',
 'Meter Consumption ID\n(Pre-filled)',
 'Portfolio Manager ID\n(Pre-filled)',
 'Property  Name\n(Pre-filled)',
 'Meter Name\n(Pre-filled)',
 'Meter Type\n(Pre-filled)',
 'Start Date\n(Required)',
 'End Date\n(Required)',
 'Quantity\n(Required)',
 'Meter Unit\n(Pre-filled)',
 'Cost\n(Optional)',
 'Estimation (Required)']

[124854000,
 5743104900,
 19516722,
 'Copy of Abbot Elementary',
 '000680140',
 'Natural Gas',
 'Last Bill 11/3/2021',
 '12/3/2021',
 118,
 'therms',
 0,
 'No']

[124854000,
 None,
 19516722,
 'Copy of Abbot Elementary',
 '000680140',
 'Natural Gas',
 None,
 None,
 None,
 'therms',
 None,
 None]

In [16]:
# this funtion deletes rows from the excel sheet that do not start with the standardized naming convention of:
# "Constellation__[CustomerId]__[MeterNumber]__[Meter Description]", with the __[Meter Description] part being optional
def delete_idxs(populated_sheet):
    delete_idxs = []
    # this is a list containing all of the indexes of rows that don't start with "Constellation__RG-"
    # we will delete all the rows with this index later
    d_idx = 1
    for row in populated_sheet.iter_rows():
        # iterate through all rows of the populated excel sheet
        meter_name = str(row[4].value)
        # extract the value from the "Meter Name (Pre-filled)" column
        if not meter_name.startswith("Constellation__RG-"):
            # if the meter name does not start with "Constellation__RG-" (for example 009300523), then we add this index to the list
            delete_idxs.append(d_idx)
        d_idx += 1
    delete_idxs.reverse()
    # now we reverse the list so we are deleting bottom-up
    delete_idxs = delete_idxs[:-1]
    # exclude the row with column headers ("Meter ID (Pre-filled)", "Meter Consumption ID (Pre-filled)", "Portfolio Manager ID, (Pre-filled)", etc.)
    # display(delete_idxs)
    for idx in delete_idxs:
        # for each index, in this list, we delete the row with that index
        populated_sheet.delete_rows(idx, amount=1)

    return

In [17]:
es_sheet = delete_idxs(es_sheet)

In [18]:
output_workbook.save(filename="Output_file.xlsx")

In [19]:
# delete_idxs = []
# d_idx = 1
# for row in es_sheet.iter_rows():
#     if not str(row[4].value).startswith("Constellation__RG-"):
#         delete_idxs.append(d_idx)
#     d_idx += 1
# delete_idxs.reverse()
# delete_idxs = delete_idxs[:-1]
# display(delete_idxs)

# for idx in delete_idxs:
#     es_sheet.delete_rows(idx, amount=1)

In [20]:
# meters_set = set()
# for df in unique_meter_data_from_const:
#     # aka for each unique meter in constellation:
#     for row in df.iterrows():
#         row_data = row[1]
#         # display(row_data)

#         # meter_num = row_data['MeterNumber']
#         # display(row_data['Const_Meter_ID'])

#         # display(type(row_data['ESPM_Meter_ID']))

#         if row_data['Const_Meter_ID'] not in energystar_data_meter_names:
#             continue
#         # else:
#         #     display(row_data)

#         # display(row_data['ESPM_Meter_ID'])

#         meter_number = row_data['MeterNumber']
#         customer_id = row_data['CustomerId']
#         meter_string = f"Constellation__{customer_id}__{meter_number}"

#         start_date = row_data['CycleStartDate']
#         end_date = row_data['CycleEndDate']
#         usage = row_data['EndReading'] - row_data['BeginningReading'] #feevolume or diff?
#         meter_multiplier = row_data['MeterMultiplier']
#         fee_volume = row_data['FeeVolume']
#         total_charges = row_data['TotalCharges']
#         actual_estimated = row_data['Actual Or Estimated']


#         insert_index = 1
#         for row in es_sheet.iter_rows():
#             # display(row[4].value)
#             if row[4].value == row_data['Const_Meter_ID']:
#                 # print("FOUND")
#                 # # sheet.insert_rows(count, amount=1)
#                 # display([cell.value for cell in row])
#                 break
#             else:
#                 insert_index += 1
        
#         insert_index += 1
#         # print("INSERT INDEX", insert_index)
#         es_sheet.insert_rows(insert_index, amount=1)

#         es_sheet.cell(row=insert_index, column=1).value = es_sheet[f"A{insert_index-1}"].value
#         # es_sheet.cell(row=insert_index, column=2).value = es_sheet[f"B{insert_index-1}"].value
#         es_sheet.cell(row=insert_index, column=3).value = es_sheet[f"C{insert_index-1}"].value
#         es_sheet.cell(row=insert_index, column=4).value = es_sheet[f"D{insert_index-1}"].value

#         # sheet.cell(row=insert_index, column=5).value = sheet[f"E{insert_index-1}"].value
#         es_sheet.cell(row=insert_index, column=5).value = meter_string
#         meters_set.add(meter_string)

#         es_sheet.cell(row=insert_index, column=6).value = es_sheet[f"F{insert_index-1}"].value
#         es_sheet.cell(row=insert_index, column=7).value = start_date
#         es_sheet.cell(row=insert_index, column=8).value = end_date
#         es_sheet.cell(row=insert_index, column=9).value = usage
#         es_sheet.cell(row=insert_index, column=10).value = es_sheet[f"J{insert_index-1}"].value
#         es_sheet.cell(row=insert_index, column=11).value = total_charges
#         es_sheet.cell(row=insert_index, column=12).value = actual_estimated

In [21]:
# final_workbook, final_sheet = create_es_excel_wb("ESPM/Empty ESPM Template.xlsx")

In [22]:
# display(final_workbook)
# display(final_sheet)
# display(final_sheet.title)

In [23]:
# max_row_input = es_sheet.max_row
# mac_col_input = es_sheet.max_column
  
# # copying the cell values from source 
# # excel file to destination excel file
# for i in range (1, max_row_input + 1):
#     for j in range (1, mac_col_input + 1):
#         # reading cell value from source excel file
#         cell = es_sheet.cell(row = i, column = j)
  
#         # writing the read value to destination excel file
#         final_sheet.cell(row = i, column = j).value = cell.value

In [24]:
# output_workbook.save(filename="ESPM/AAPS_Cleaned_Rows2.xlsx")

In [25]:
# es_sheet.insert_rows(insert_index, amount=1)

In [26]:
# x = 'Constellation__RG-43035769__2420793405'
# x.startswith("Constellation__RG")

In [27]:
# es_sheet.insert_rows(insert_index, amount=1)

In [28]:
# this section was old from the fake Rosetta Stone Holy Grail Bible file:

# const_to_portfolio_id = pd.read_csv("Consetllation/const_to_portfolio_id.tsv", header=None, sep='\t')
# const_to_portfolio_id.columns = ["Constellation_Customer_ID_Meter_ID", "ESPM_Meter_ID"]

# const_to_portfolio_id

# const_to_portfolio_id['Meter_ID'] = const_to_portfolio_id['Constellation_Customer_ID_Meter_ID'].str.split("__", expand=True)[2]

# const_to_portfolio_id.dtypes

# const_to_portfolio_id['Meter_ID'] = const_to_portfolio_id['Meter_ID'].astype("int")

# const_to_portfolio_id['Meter_ID'] = pd.to_numeric(const_to_portfolio_id['Meter_ID'])

# const_to_portfolio_id.dtypes

# const_to_portfolio_id

# merged_constellation = pd.merge(constellation_excel, const_to_portfolio_id, left_on="MeterNumber", right_on="Meter_ID")

# merged_constellation

# unique_espm_meter_ids_from_const = []

# for unique_meter_name in merged_constellation['ESPM_Meter_ID'].unique():
#     display(unique_meter_name)
#     temp_df = merged_constellation[merged_constellation['ESPM_Meter_ID'] == unique_meter_name]
#     # display(temp_df)
#     unique_espm_meter_ids_from_const.append(temp_df)

In [29]:
# unique_meter_data_from_const = []

# for unique_meter_name in constellation_excel['MeterNumber'].unique():
#     # display(unique_meter_name)
#     temp_df = constellation_excel[constellation_excel['MeterNumber'] == unique_meter_name]
#     # display(temp_df)
#     # if unique_meter_name.startswith("Constellation__RG"):   
#     unique_meter_data_from_const.append(temp_df)
#         # unique_meter_names_from_const.append(unique_meter_name)


# unique_meter_data_from_const[0]
# # Meter Name\n(Pre-filled)
# # unique_espm_meter_ids_from_const

In [30]:
# constellation_excel.columns

# def name_helper(row):
#     row_id = str(row['CustomerId'])
#     row_number = str(row['MeterNumber'])
#     final_string = f"Constellation__{row_id}__{row_number}"
#     return final_string

# constellation_excel['Const_Meter_ID'] = constellation_excel.apply(name_helper, axis=1)
# constellation_excel

In [31]:
# from openpyxl import Workbook
# from openpyxl.utils.dataframe import dataframe_to_rows
# from openpyxl import load_workbook

# output_workbook = load_workbook(filename="ESPM/Test_3_8_22/Add_Bills_to_Meters NEW Naming AAPS Meters.xlsx")
# display(output_workbook.sheetnames)

# sheet = output_workbook.active
# display(sheet)
# display(sheet.title)

# for row in sheet.iter_rows():
#     print(row[4].value)

# for row in sheet.iter_rows( max_row=6):
#     display([cell.value for cell in row])

In [32]:
# energystar_data.columns = energystar_data.columns.str.replace('\n', ' ')
# display(energystar_data.columns)
# display(energystar_data)
# # energystar_data_meter_names = energystar_data["Meter Name (Pre-filled)"]
# energystar_data["Meter Name (Pre-filled)"] = energystar_data["Meter Name (Pre-filled)"].str.split(",").str[0]
# energystar_data["Meter Name (Pre-filled)"] = energystar_data["Meter Name (Pre-filled)"][energystar_data["Meter Name (Pre-filled)"].str.contains("Constellation__RG-\d{8}__\d{10}") == True]
# energystar_data_meter_names = energystar_data["Meter Name (Pre-filled)"].unique()
# display(energystar_data_meter_names)

In [33]:
# energystar_excel = pd.read_excel("ESPM/Test_3_8_22/Add_Bills_to_Meters NEW Naming AAPS Meters.xlsx", sheet_name=None)
# # energystar_excel = pd.read_excel("ESPM/AAPS Upload TemplateTest_2-17-2022.xlsx", sheet_name=None)
# display(energystar_excel.keys())
# print("LENGTH: ", len(energystar_excel.items()))
# energystar_data = energystar_excel['Add Bills-Non Electric']
# display(energystar_data)